***
<center><h1 style="color:#28abb9;font-weight:bold;">Neighborhoods data cleaning<h1/><center/>

***

In [11]:
import lxml.html as lh
import pandas as pd
import numpy as np
print('imports completed')

imports completed


# Download wikipedia page

In [12]:
!wget wiki.txt https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('download complete')

--2021-01-01 18:49:39--  http://wiki.txt/
Resolving wiki.txt (wiki.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘wiki.txt’
--2021-01-01 18:49:39--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 91.198.174.192, 2620:0:862:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|91.198.174.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54812 (54K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

List_of_postal_code 100%[===================>]  53.53K  --.-KB/s    in 0.1s    

2021-01-01 18:49:40 (431 KB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [54812/54812]

FINISHED --2021-01-01 18:49:40--
Total wall clock time: 0.7s
Downloaded: 1 files, 54K in 0.1s (431 KB/s)
download complete


# Extracting rows and columns

In [13]:
file=open('List_of_postal_codes_of_Canada:_M','r')
content=file.read()
table_data=lh.fromstring(content).xpath('//tr')
row_size=3
#extracting columns
columns=[]
i=0
for col in table_data[0]:
    i=i+1
    columns.append(col.text_content())
columns
#reformating columns array
j=0
for col in columns:
    columns[j]=col[:col.find('\n')]
    j=j+1
columns
rows=[]
#Extracting rows
for i in range(1,len(table_data)):
    row=table_data[i]
    if len(row)!=3:
        break
    rows.append(row.text_content())

for i,row in enumerate(rows):
    rows[i]=row[1:len(row)-1].split('\n\n')
rows[:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']]

# Dataframe creation

In [14]:
df=pd.DataFrame(data=rows,columns=columns,index=None)
df.head()
# removing null values
df.loc[df['Borough']=='Not assigned']=np.nan
df.dropna(axis=0,inplace=True)
df.reset_index()


,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [15]:
df.loc[df['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


# Editing not assigned neighbourhood


In [16]:
for index,row in df.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']=row['Borough']

df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [17]:
df.shape

(104, 3)

# LOading geospatial data

In [18]:
!wget https://cocl.us/Geospatial_data

--2021-01-01 18:49:41--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-01 18:49:42--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-01 18:49:43--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]


In [19]:
df_geo=pd.read_csv('Geospatial_data')
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [20]:
for i,row_g in df_geo.iterrows():
    for j,row in df.iterrows():
        if (row['Postal Code']==row_g['Postal Code']):
            df.loc[df['Postal Code']==row['Postal Code'],['Latitude']]=row_g['Latitude']
            df.loc[df['Postal Code']==row['Postal Code'],['Longitude']]=row_g['Longitude']
df 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999
